In [71]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
print("tensorflow version {}".format(tf.VERSION)) # make sure its tensorflow 1.*.*

tensorflow version 1.12.0


In [72]:
#c=a+b
#d=c*a
g0=tf.Graph()
with g0.as_default(): # context manager
    a = tf.constant([[1],[2]], dtype=tf.float32,name='a_constant')
    b = tf.constant([[2],[3]], dtype=tf.float32,name='b_constant')
    c=tf.add(a,b,name='add')  # c = a + b
    d=c * a #d = c * a
print(a)
print(b)
print(c)
print(d)
#building the graph (the actual computation does not happen here, though its tempting to think of that)
#build the engine

Tensor("a_constant:0", shape=(2, 1), dtype=float32)
Tensor("b_constant:0", shape=(2, 1), dtype=float32)
Tensor("add:0", shape=(2, 1), dtype=float32)
Tensor("mul:0", shape=(2, 1), dtype=float32)


In [73]:
#computation occurs
#start the engine
sess0=tf.Session(graph=g0)
check = ['a','c','d']
result=sess0.run([a,c,d])
for i, re in zip(check,result):
    print('value of {} =\n {}'.format(i,re))

#visualize the graph
writer = tf.summary.FileWriter('.',sess0.graph)
writer.flush()

sess0.close()

value of a =
 [[1.]
 [2.]]
value of c =
 [[3.]
 [5.]]
value of d =
 [[ 3.]
 [10.]]


In [74]:
!pwd

/Users/haobei/Projects/Tutorial/L3


In [75]:
#quiz 1
#g0_q=tf.Graph()
#c = a @ b
#a.shape=(4,2), b.shape=(2,3)
g0_q=tf.Graph()
with g0_q.as_default():
    a = tf.constant(tf.ones(4,2),dtype=tf.float32)
    b = tf.constant(tf.ones(2,3),dtype=tf.float32)
    c = a @ b
sess0q=g

TypeError: Expected float32, got list containing Tensors of type '_Message' instead.

In [76]:
#tensor math matrix
#with "defaultGraph".as_default():
g1=tf.Graph()
a0 = tf.constant(2)
with g1.as_default():
    a = tf.random_normal([2,3], dtype=tf.float32,name='a_random')
    b = tf.constant([[1,1],[1,1],[1,1]], dtype=tf.float32,name='b')
    op = tf.print("value of a",a,name='print')
    var_x = tf.get_variable("var_x", [2,2], dtype=tf.float32, trainable=True,
                            initializer=tf.glorot_normal_initializer)
    c = tf.matmul(a,b,name='matmul') # a @ b
    d = c + var_x #tf.add
print(a0,a0.graph)
print(a,a.graph)
print(b,b.graph)
print(op,op.graph)
print(var_x,var_x.graph)
print(c,c.graph)
print(d,d.graph)

Tensor("Const_1:0", shape=(), dtype=int32) <tensorflow.python.framework.ops.Graph object at 0xb3f03c0b8>
Tensor("a_random:0", shape=(2, 3), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3f08a3c8>
Tensor("b:0", shape=(3, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3f08a3c8>
name: "print"
op: "PrintV2"
input: "print_format"
attr {
  key: "output_stream"
  value {
    s: "stderr"
  }
}
<tf.Variable 'var_x:0' shape=(2, 2) dtype=float32_ref> <tensorflow.python.framework.ops.Graph object at 0xb3f08a3c8>
Tensor("matmul:0", shape=(2, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3f08a3c8>
Tensor("add:0", shape=(2, 2), dtype=float32) <tensorflow.python.framework.ops.Graph object at 0xb3f08a3c8>


In [77]:
#engine switch
sess1=tf.Session(graph=g1)

In [78]:
print('print',sess1.run(op))
print('a',sess1.run(a)) #print(a.eval(session=sess))
print('matmul',sess1.run('matmul'))
print('c',sess1.run(c)) #c
print('a,c',sess1.run([a,c,a]))
# print('initilizer',sess1.run(var_x.initializer))
with g1.as_default():
    sess1.run(tf.global_variables_initializer())
print('d',sess1.run(d))
print('var_x',sess1.run(var_x))
print('d',sess1.run(d))
print('assign',sess1.run(var_x.assign_add([[1,0],[1,1]])))
print('var_x,d',sess1.run([var_x,d]))

print None
a [[ 0.49499008  1.9555719  -0.7545054 ]
 [ 0.45289025 -1.7964365   0.3596094 ]]
matmul None
c [[-0.8689999 -0.8689999]
 [ 1.8697393  1.8697393]]
a,c [array([[-0.06084506,  0.36236683,  0.27671674],
       [ 0.4164595 , -1.4819223 , -0.2758948 ]], dtype=float32), array([[ 0.5782385,  0.5782385],
       [-1.3413576, -1.3413576]], dtype=float32), array([[-0.06084506,  0.36236683,  0.27671674],
       [ 0.4164595 , -1.4819223 , -0.2758948 ]], dtype=float32)]
d [[-5.1564226  -5.419283  ]
 [ 0.28008488  0.34103048]]
var_x [[ 0.12384737 -0.13901274]
 [ 0.33427343  0.39521903]]
d [[ 2.6236043  2.360744 ]
 [-0.3076128 -0.2466672]]
assign [[ 1.1238474  -0.13901274]
 [ 1.3342735   1.3952191 ]]
var_x,d [array([[ 1.1238474 , -0.13901274],
       [ 1.3342735 ,  1.3952191 ]], dtype=float32), array([[2.4343586 , 1.1714984 ],
       [0.45895058, 0.5198962 ]], dtype=float32)]


In [79]:
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess1.graph)
writer.flush()
tf.reset_default_graph()
v = tf.get_variable("d",shape=[2,3],dtype=tf.float32,initializer=tf.glorot_normal_initializer)

In [80]:
#facilitate updates of weights of a neural network
with g1.as_default():
    sess1.run(tf.global_variables_initializer())
    print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)) 
    print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

[<tf.Variable 'var_x:0' shape=(2, 2) dtype=float32_ref>]
[<tf.Variable 'var_x:0' shape=(2, 2) dtype=float32_ref>]
[<tf.Variable 'd:0' shape=(2, 3) dtype=float32_ref>]


In [81]:
sess1.close()

In [83]:
#quiz 2
#g1_q=tf.Graph()
#d = tf.reduce_sum(var_w * ((L @ x)^2)) + var_b
# var_w.shape=(2,) L.shape=(2,3) x.shape=(3,1) var_b.shape=() initializer: tf.random_normal_initializer
g1_q=tf.Graph()
with g1_q.as_default():
    x = tf.constant([[0],[1],[2]],dtype=tf.float32)
    var_w = tf.get_variable('var_w',shape=(2,1),dtype=tf.float32,initializer=tf.ones_initializer)
    L = tf.constant([[0,1,2],[2,3,4]],name='L',dtype=tf.float32)
    var_b = tf.get_variable('var_b',shape=(),dtype=tf.float32,initializer=tf.ones_initializer)
    d=tf.reduce_sum(var_w * tf.square(L @ x))+var_b

In [84]:
sess_g1_q = tf.Session(graph=g1_q)
# sess_g1_q.run([var_w.initializer,var_b.initializer])
with g1_q.as_default():
    sess_g1_q.run(tf.global_variables_initializer())
print(sess_g1_q.run(d))
writer = tf.summary.FileWriter('.',sess_g1_q.graph)
writer.flush()

147.0


In [96]:
x = np.array([[1], [2], [3], [4]],dtype=np.float32)
y = np.array([[4], [3], [2], [1]],dtype=np.float32)

In [97]:
# build default graph
tf.reset_default_graph()
# mini-batch 
x_p = tf.placeholder(dtype=tf.float32,shape=(None,1),name="x_placeholder") # shape is determained at runtime
y_p = tf.placeholder(dtype=tf.float32,shape=(None,1),name="y_placeholder")

W = tf.get_variable("weight",shape=(),dtype=tf.float32,
                    initializer=tf.ones_initializer,
                    trainable=True)
b = tf.get_variable("bias",shape=(),dtype=tf.float32,
                    initializer=tf.zeros_initializer,
                    trainable=True)
y_hat = W * x_p + b
loss = tf.reduce_mean(tf.square(y_hat - y_p))

In [98]:
sess=tf.Session() # session of the default graph
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess.graph)
writer.flush()

In [99]:
gradient = tf.gradients(loss,[W,b])
#print the design blue print of the engine
writer = tf.summary.FileWriter('.',sess.graph)
writer.flush()

In [106]:
sess.run(tf.global_variables_initializer())
W_init = sess.run([W,b])
print(W_init)
prediction = sess.run([y_hat,loss,gradient],feed_dict={x_p: x,y_p: y})
print(prediction)
for i in range(100):
    grad=sess.run(gradient,feed_dict={x_p: x,y_p: y})
    W_old,b_old = sess.run([W,b])
    sess.run([W.assign_sub(0.1*grad[0]),b.assign_sub(0.1*grad[1])]) #what about millions of weights?
#     sess.run([var.assign_sub(0.1*gradient) for var,gradient in zip(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES),grad)])
    W_new,b_new = sess.run([W,b])
    print(i,"gradient: ",grad, "\t", W_old,b_old, "==>",W_new,b_new)
print(sess.run([W,b]))

[1.0, 0.0]
[array([[1.],
       [2.],
       [3.],
       [4.]], dtype=float32), 5.0, [5.0, 0.0]]
0 gradient:  [5.0, 0.0] 	 1.0 0.0 ==> -4.0 0.0
1 gradient:  [-70.0, -25.0] 	 -4.0 0.0 ==> 66.0 25.0
2 gradient:  [1105.0, 375.0] 	 66.0 25.0 ==> -1039.0 -350.0
3 gradient:  [-17345.0, -5900.0] 	 -1039.0 -350.0 ==> 16306.0 5550.0
4 gradient:  [272330.0, 92625.0] 	 16306.0 5550.0 ==> -256024.0 -87075.0
5 gradient:  [-4275745.0, -1454275.0] 	 -256024.0 -87075.0 ==> 4019721.0 1367200.0
6 gradient:  [67131810.0, 22833000.0] 	 4019721.0 1367200.0 ==> -63112090.0 -21465800.0
7 gradient:  [-1054010300.0, -358492030.0] 	 -63112090.0 -21465800.0 ==> 990898240.0 337026240.0
8 gradient:  [16548604000.0, 5628544000.0] 	 990898240.0 337026240.0 ==> -15557706000.0 -5291518000.0
9 gradient:  [-259823170000.0, -88371560000.0] 	 -15557706000.0 -5291518000.0 ==> 244265470000.0 83080045000.0
10 gradient:  [4079382200000.0, 1387487400000.0] 	 244265470000.0 83080045000.0 ==> -3835116700000.0 -1304407400000.0
1

In [105]:
prediction = sess.run([y_hat,loss,gradient],feed_dict={x_p: x,y_p: y})
print("prediction")
print(prediction[0])
print("target")
print(y)
# 1000 steps

prediction
[[1.5107338]
 [1.7937801]
 [2.0768263]
 [2.3598723]]
target
[[4.]
 [3.]
 [2.]
 [1.]]


In [30]:
# single sigmoid neuron prediction on two-dimension data
x = np.array([[0,0],[1,0],[0,1],[1,1]])
y = np.array([[0],[0],[1],[1]])
#tf.sigmoid() tf.reduce_mean()

In [ ]:
#your implementation goes here, add more code cells if necessary

In [ ]:
# visualize the model
vis_inputs = np.random.uniform(size=(10000,2))
predictions = sess.run(y_hat,feed_dict={x_p:vis_inputs})
print(predictions.shape)
colors = list(map(lambda p: 'red' if p < 0.5 else 'blue', predictions))
plt.scatter(vis_inputs[:,0], vis_inputs[:,1], 1, colors)

In [ ]:
# single sigmoid neuron prediction on () two-dimension data
x = np.array([[0,0],[1,0],[0,1],[1,1]])
y = np.array([[0],[1],[1],[0]])

In [ ]:
# visualize the model
vis_inputs = np.random.uniform(size=(10000,2))
predictions = sess.run(y_hat,feed_dict={x_p:vis_inputs})
print(predictions.shape)
colors = list(map(lambda p: 'red' if p < 0.5 else 'blue', predictions))
plt.scatter(vis_inputs[:,0], vis_inputs[:,1], 1, colors)

# Summary
## 1. What is tensorflow?
## 2. Why do people use tensorflow?
## 3. How to write tensorflow program?
### (1). Build dataflow graph
### (2). Run dataflow graph and train the weights
## 4. (Bonus) What is feature learning and feature crafting (feature engineering), Difference, how to do feature crafting?

## 1. 

## 2.

## 3.(1). 

## 3.(2).

## 4.